# Data Scraping - LinkedIn data

In [85]:
from selenium import webdriver
import time
import pandas as pd
import csv
import re
import spacy
import sys
from importlib import reload
reload(sys)
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import sys, getopt
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from sklearn.feature_extraction.text import TfidfVectorizer
stop_words = set(stopwords.words('english'))

In [88]:
import warnings
warnings.filterwarnings('ignore')

In [90]:
url ="https://www.linkedin.com/jobs/Engineering-intern-jobs?position=1&pageNum=0"
wd = webdriver.Chrome(executable_path="C:/Users/messi/OneDrive/Desktop/Desktop/chromedriver.exe")
wd.get(url)


page_num = 0

no_of_jobs = int(str(wd.find_element_by_css_selector('h1>span').get_attribute('innerText')).replace('+', '').replace(',', ''))

#print(no_of_jobs)
if no_of_jobs > 150 :
    no_of_jobs = 150

for i in range(6):
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i = i + 1
    try:
        wd.find_element_by_xpath('/html/body/main/div/section/button').click()
        wd.find_element_by_class_name('infinite-scroller__show-more-button.infinite').click()
        if(wd.find_element_by_class_name('infinite-scroller__show-more-button.infinite')):
            page_num = page_num + 1
        time.sleep(0.5)
    except:
        pass
        time.sleep(0.5)
#print("page_num:",page_num)

    
job_lists = wd.find_element_by_class_name('jobs-search__results-list')
jobs = job_lists.find_elements_by_tag_name('li')


job_id= []
job_title = []
company_name = []
location = []
date = []
job_link = []


#for all the job take job data - Title, job_id, link, posted date, job location
for job in jobs:
        job_id0 = str(job.find_element_by_css_selector('div').get_attribute('data-entity-urn')).replace("urn:li:jobPosting:", "")
        job_id.append(job_id0)


        job_title0 = job.find_element_by_css_selector('h3').get_attribute('innerText')
        job_title.append(job_title0)

        company_name0 = job.find_element_by_css_selector('h4').get_attribute('innerText')
        company_name.append(company_name0)

        location0 = job.find_element_by_css_selector('[class="job-search-card__location"]').get_attribute('innerText')
        location.append(location0)

        date0 = job.find_element_by_css_selector('div>div>time').get_attribute('datetime')
        date.append(date0)

        job_link0 = job.find_element_by_css_selector('a').get_attribute('href')
        job_link.append(job_link0)
jd = []

# We are trying open a new tab for each job link to scrape detailed job decription
for i in range(len(job_link)):
    try:
        wd1 = webdriver.Chrome(executable_path="C:/Users/messi/OneDrive/Desktop/Desktop/chromedriver.exe") 
        wd1.get(job_link[i])
        jd0= wd1.find_element_by_css_selector('[class="show-more-less-html__markup show-more-less-html__markup--clamp-after-5"]').get_attribute('innerText')
        jd.append(jd0)
        time.sleep(0.2)
    except:
        wd1.get(job_link[i])
        jd0= wd1.find_element_by_css_selector('[class="show-more-less-html__markup show-more-less-html__markup--clamp-after-5"]').get_attribute('innerText')
        jd.append(jd0)
        time.sleep(0.2)
    wd1.quit()

#print(len(jd)) 


# Store it in Dataframe and write it in excel file
job_data = pd.DataFrame({'ID': job_id,'Date': date,'Company': company_name,'Title': job_title,'Location': location, 'Link': job_link, 'Job Description': jd})
job_data.to_excel('LinkedIn Job.xlsx', index = False)

print("Data Scraped Successfully")

Data Scraped Successfully
